## 🧩 Part 1 — LangChain Core Concepts (Lecture 4.1)

**Goal:** Get familiar with the LangChain components and connect them to real data.

### Step 1: Install Dependencies

## 🚀 Quick Start Guide

**To run this notebook successfully:**

1. **Run Cell 2** - Install basic packages
2. **Run Cell 4** - Install additional LangChain packages  
3. **Create a `.env` file** in your workspace root with: `OPENROUTER_API_KEY=your_key_here`
4. **Restart the kernel** (to clear any import cache)
5. **Run Cell 6** - Import libraries (should work without errors now!)
6. **Continue running cells sequentially**

---

In [ ]:
# Install required packages one by one for clarity
%pip install langchain
%pip install langchain-community
%pip install langchain-openai
%pip install openai
%pip install faiss-cpu
%pip install tiktoken
%pip install python-dotenv

**💡 Important Note:** The next cell installs the **FULL `langchain` package** which includes the `chains` module. This is essential for `RetrievalQA` to work. The modular packages alone (`langchain-core`, `langchain-community`) don't include chains functionality.

In [ ]:
# Install the FULL langchain package (includes chains module)
# This is the KEY package that was missing!
%pip install --upgrade langchain

# Also install these supporting packages
%pip install langchain-core
%pip install langchain-text-splitters

### Step 2: Import Core Modules and Configure OpenRouter API

**Important:** Make sure you have your OpenRouter API key set in a `.env` file:
```
OPENROUTER_API_KEY=your_key_here
```

Get your free API key from [https://openrouter.ai/keys](https://openrouter.ai/keys)

In [21]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key from environment
api_key = os.getenv("OPENROUTER_API_KEY")

if not api_key:
    raise ValueError("⚠️ OPENROUTER_API_KEY not found! Please set it in your .env file")

# Configure OpenRouter for both embeddings and chat
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

print("✅ Libraries imported and OpenRouter configured successfully!")

✅ Libraries imported and OpenRouter configured successfully!


**🔧 Note on LangChain Version:** This notebook uses **LangChain 1.2.0+**, which has a different API than older versions. The `RetrievalQA` class has been replaced with newer chain construction methods. We've created helper functions to maintain the same functionality.

In [22]:
# Helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Helper function to create a QA chain
def create_qa_chain(llm, retriever):
    prompt = ChatPromptTemplate.from_template(
        "Answer the question based only on the following context:\n\n{context}\n\nQuestion: {question}\n\nAnswer:"
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # Wrapper to return source documents
    def qa_with_sources(query):
        docs = retriever.invoke(query)
        answer = rag_chain.invoke(query)
        return {"result": answer, "source_documents": docs}
    
    return qa_with_sources

print("✅ Helper functions created!")

✅ Helper functions created!


### Step 3: Create Sample Documents

These documents will form the knowledge base for our RAG system.

In [23]:
texts = [
    "Retrieval-Augmented Generation (RAG) enhances LLM accuracy by grounding responses in real data.",
    "LangChain provides modular tools to connect LLMs with external knowledge sources.",
    "FAISS is used for fast vector search and similarity-based retrieval.",
    "OpenRouter provides unified access to multiple LLM providers through a single API.",
    "Vector databases store embeddings and enable semantic search capabilities.",
    "Text chunking is essential for managing context windows in LLMs."
]

docs = [Document(page_content=t) for t in texts]
print(f"✅ Created {len(docs)} documents")
for i, doc in enumerate(docs):
    print(f"{i+1}. {doc.page_content[:60]}...")

✅ Created 6 documents
1. Retrieval-Augmented Generation (RAG) enhances LLM accuracy b...
2. LangChain provides modular tools to connect LLMs with extern...
3. FAISS is used for fast vector search and similarity-based re...
4. OpenRouter provides unified access to multiple LLM providers...
5. Vector databases store embeddings and enable semantic search...
6. Text chunking is essential for managing context windows in L...


### Step 4: Chunk Data for Embedding

Splitting text into smaller chunks ensures better retrieval accuracy and manages context window limits.

In [24]:
splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
chunks = splitter.split_documents(docs)

print(f"✅ Total chunks created: {len(chunks)}")
print("\nFirst 3 chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1}:")
    print(f"Content: {chunk.page_content}")
    print(f"Length: {len(chunk.page_content)} characters")

✅ Total chunks created: 6

First 3 chunks:

Chunk 1:
Content: Retrieval-Augmented Generation (RAG) enhances LLM accuracy by grounding responses in real data.
Length: 95 characters

Chunk 2:
Content: LangChain provides modular tools to connect LLMs with external knowledge sources.
Length: 81 characters

Chunk 3:
Content: FAISS is used for fast vector search and similarity-based retrieval.
Length: 68 characters


**✅ Output:** You should see multiple small text chunks ready for vectorization — these are the atomic units RAG retrieves later.

---

## ⚙️ Part 2 — RAG Implementation with LangChain (Lecture 4.2)

**Goal:** Build a functional RAG pipeline: Embed → Store → Retrieve → Generate.

### Step 1: Create Embeddings and Store in FAISS

We'll use OpenRouter's embedding endpoint (OpenAI-compatible) to generate vector embeddings.

In [ ]:
# Initialize embeddings using OpenRouter
embeddings = OpenAIEmbeddings(
    model="openai/text-embedding-3-small",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=api_key
)

# Create FAISS vectorstore from documents
print("🔄 Creating embeddings and building FAISS index...")
vectorstore = FAISS.from_documents(chunks, embeddings)

# Create retriever with similarity search
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 2}  # Retrieve top 2 most relevant chunks
)

print("✅ Vectorstore created and retriever configured!")
print(f"📊 Index contains {vectorstore.index.ntotal} vectors")


🔄 Creating embeddings and building FAISS index...
✅ Vectorstore created and retriever configured!
📊 Index contains 6 vectors


### Step 2: Initialize LLM

We'll use OpenRouter to access various LLM models. Here we're using a fast and capable model.

In [26]:
# Initialize ChatOpenAI with OpenRouter
llm = ChatOpenAI(
    model_name="meta-llama/llama-3.1-8b-instruct",  # Fast, capable model
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=api_key,
    temperature=0.3  # Lower temperature for more focused responses
)

print("✅ LLM initialized successfully!")
print(f"📋 Model: meta-llama/llama-3.1-8b-instruct")

✅ LLM initialized successfully!
📋 Model: meta-llama/llama-3.1-8b-instruct


### Step 3: Build the RAG Chain

The RetrievalQA chain combines the retriever and LLM to create a question-answering system.

In [27]:
# Build the RAG chain
qa_chain = create_qa_chain(llm, retriever)

print("✅ RAG chain created successfully!")
print("🎯 Ready to answer questions!")

✅ RAG chain created successfully!
🎯 Ready to answer questions!


### Step 4: Ask Your RAG System a Question

Let's test the pipeline with a query about LangChain.

In [28]:
query = "What is the benefit of using LangChain in RAG?"

print(f"❓ Query: {query}\n")
print("🔄 Processing...")

result = qa_chain(query)

print("\n" + "="*70)
print("💡 ANSWER:")
print("="*70)
print(result["result"])

print("\n" + "="*70)
print("📚 CONTEXT USED:")
print("="*70)
for i, doc in enumerate(result["source_documents"]):
    print(f"\nSource {i+1}:")
    print(doc.page_content)

❓ Query: What is the benefit of using LangChain in RAG?

🔄 Processing...

💡 ANSWER:
The benefit of using LangChain in RAG is that it enhances LLM accuracy by grounding responses in real data.

📚 CONTEXT USED:

Source 1:
LangChain provides modular tools to connect LLMs with external knowledge sources.

Source 2:
Retrieval-Augmented Generation (RAG) enhances LLM accuracy by grounding responses in real data.


**✅ Expected Output:** A concise, context-grounded answer referencing LangChain's modular structure and role in retrieval workflows.

### Step 5: Test with Another Query

In [29]:
query2 = "How does FAISS help in RAG systems?"

print(f"❓ Query: {query2}\n")
result2 = qa_chain(query2)

print("\n" + "="*70)
print("💡 ANSWER:")
print("="*70)
print(result2["result"])

print("\n" + "="*70)
print("📚 CONTEXT USED:")
print("="*70)
for i, doc in enumerate(result2["source_documents"]):
    print(f"\nSource {i+1}:")
    print(doc.page_content)

❓ Query: How does FAISS help in RAG systems?


💡 ANSWER:
FAISS helps in RAG systems by enabling fast vector search and similarity-based retrieval, which is crucial for efficiently retrieving relevant data to ground responses in real data.

📚 CONTEXT USED:

Source 1:
FAISS is used for fast vector search and similarity-based retrieval.

Source 2:
Retrieval-Augmented Generation (RAG) enhances LLM accuracy by grounding responses in real data.


---

## 🧠 Part 3 — Adding Context and Metadata (Lecture 4.3)

**Goal:** Enhance the pipeline by attaching metadata such as document source, category, or author — and show how it improves retrieval context.

### Step 1: Rebuild Documents with Metadata

Metadata enriches your knowledge base with provenance information.

In [30]:
# Create documents with rich metadata
docs_with_metadata = [
    Document(
        page_content="LangChain helps connect LLMs with external knowledge bases.",
        metadata={"source": "LangChain Docs", "category": "Framework", "date": "2024"}
    ),
    Document(
        page_content="RAG systems improve factual accuracy by combining retrieval and generation.",
        metadata={"source": "OpenAI Blog", "category": "AI Research", "date": "2023"}
    ),
    Document(
        page_content="FAISS allows efficient vector similarity search for embeddings.",
        metadata={"source": "Meta Research", "category": "Database", "date": "2024"}
    ),
    Document(
        page_content="OpenRouter provides a unified API to access multiple AI models from different providers.",
        metadata={"source": "OpenRouter Docs", "category": "Platform", "date": "2024"}
    ),
    Document(
        page_content="Embeddings transform text into dense vector representations that capture semantic meaning.",
        metadata={"source": "AI Fundamentals", "category": "AI Research", "date": "2023"}
    ),
    Document(
        page_content="Text chunking strategies significantly impact RAG system performance and accuracy.",
        metadata={"source": "RAG Best Practices", "category": "Tutorial", "date": "2024"}
    )
]

print(f"✅ Created {len(docs_with_metadata)} documents with metadata\n")
print("Sample document with metadata:")
print(f"Content: {docs_with_metadata[0].page_content}")
print(f"Metadata: {docs_with_metadata[0].metadata}")

✅ Created 6 documents with metadata

Sample document with metadata:
Content: LangChain helps connect LLMs with external knowledge bases.
Metadata: {'source': 'LangChain Docs', 'category': 'Framework', 'date': '2024'}


### Step 2: Recreate the FAISS Vectorstore with Metadata

In [31]:
# Recreate vectorstore with metadata-enriched documents
print("🔄 Creating new vectorstore with metadata...")
vectorstore_with_metadata = FAISS.from_documents(docs_with_metadata, embeddings)

retriever_with_metadata = vectorstore_with_metadata.as_retriever(
    search_kwargs={"k": 2}
)

# Rebuild the QA chain with the new retriever
qa_chain_with_metadata = create_qa_chain(llm, retriever_with_metadata)

print("✅ Vectorstore and chain recreated with metadata!")

🔄 Creating new vectorstore with metadata...
✅ Vectorstore and chain recreated with metadata!


### Step 3: Query with Context and Display Metadata

Now let's ask a question and see how metadata enriches the response context.

In [32]:
query3 = "Which tool helps with efficient vector search?"

print(f"❓ Query: {query3}\n")
result3 = qa_chain_with_metadata(query3)

print("="*70)
print("💡 ANSWER:")
print("="*70)
print(result3["result"])

print("\n" + "="*70)
print("📚 METADATA FROM RETRIEVED DOCS:")
print("="*70)
for i, doc in enumerate(result3["source_documents"]):
    print(f"\n🔹 Document {i+1}:")
    print(f"   Content: {doc.page_content}")
    print(f"   Source: {doc.metadata['source']}")
    print(f"   Category: {doc.metadata['category']}")
    print(f"   Date: {doc.metadata['date']}")

❓ Query: Which tool helps with efficient vector search?

💡 ANSWER:
FAISS

📚 METADATA FROM RETRIEVED DOCS:

🔹 Document 1:
   Content: FAISS allows efficient vector similarity search for embeddings.
   Source: Meta Research
   Category: Database
   Date: 2024

🔹 Document 2:
   Content: Embeddings transform text into dense vector representations that capture semantic meaning.
   Source: AI Fundamentals
   Category: AI Research
   Date: 2023


**✅ Expected Output:** Answer mentions "FAISS," with metadata showing it came from the "Meta Research" source.

### Step 4: Advanced Query with Metadata Context

In [33]:
query4 = "What are the key components of a RAG system?"

print(f"❓ Query: {query4}\n")
result4 = qa_chain_with_metadata(query4)

print("="*70)
print("💡 ANSWER:")
print("="*70)
print(result4["result"])

print("\n" + "="*70)
print("📚 SOURCES AND METADATA:")
print("="*70)
for i, doc in enumerate(result4["source_documents"]):
    print(f"\n🔹 Source {i+1}: {doc.metadata['source']} ({doc.metadata['category']})")
    print(f"   Content: {doc.page_content}")
    print(f"   Published: {doc.metadata['date']}")

❓ Query: What are the key components of a RAG system?

💡 ANSWER:
Retrieval and Generation.

📚 SOURCES AND METADATA:

🔹 Source 1: OpenAI Blog (AI Research)
   Content: RAG systems improve factual accuracy by combining retrieval and generation.
   Published: 2023

🔹 Source 2: RAG Best Practices (Tutorial)
   Content: Text chunking strategies significantly impact RAG system performance and accuracy.
   Published: 2024


### Step 5: Test Retrieval by Metadata Category

Let's see what documents we have in each category.

In [34]:
# Analyze metadata distribution
from collections import defaultdict

category_docs = defaultdict(list)
for doc in docs_with_metadata:
    category_docs[doc.metadata['category']].append(doc.page_content[:50] + "...")

print("📊 Documents by Category:\n")
for category, contents in category_docs.items():
    print(f"🏷️  {category}:")
    for content in contents:
        print(f"   • {content}")
    print()

📊 Documents by Category:

🏷️  Framework:
   • LangChain helps connect LLMs with external knowled...

🏷️  AI Research:
   • RAG systems improve factual accuracy by combining ...
   • Embeddings transform text into dense vector repres...

🏷️  Database:
   • FAISS allows efficient vector similarity search fo...

🏷️  Platform:
   • OpenRouter provides a unified API to access multip...

🏷️  Tutorial:
   • Text chunking strategies significantly impact RAG ...



---

## 🧭 Bonus Challenges

Try these extensions to deepen your understanding:

### Challenge 1: Add New Document Category

In [35]:
# Add documents with "Use Cases" category
new_docs = [
    Document(
        page_content="E-commerce companies use RAG to provide accurate product recommendations based on inventory data.",
        metadata={"source": "Industry Report", "category": "Use Cases", "date": "2024"}
    ),
    Document(
        page_content="Healthcare providers leverage RAG systems to query medical knowledge bases for diagnosis support.",
        metadata={"source": "Medical AI Journal", "category": "Use Cases", "date": "2024"}
    ),
    Document(
        page_content="Legal firms employ RAG to search and analyze case law and legal precedents efficiently.",
        metadata={"source": "Legal Tech Review", "category": "Use Cases", "date": "2024"}
    )
]

# Combine with existing documents
all_docs = docs_with_metadata + new_docs

# Recreate vectorstore
print("🔄 Adding new 'Use Cases' category documents...")
vectorstore_extended = FAISS.from_documents(all_docs, embeddings)
retriever_extended = vectorstore_extended.as_retriever(search_kwargs={"k": 3})

qa_chain_extended = create_qa_chain(llm, retriever_extended)

print(f"✅ Extended vectorstore now contains {len(all_docs)} documents!")

# Test with use-case query
query5 = "What are some real-world applications of RAG systems?"
print(f"\n❓ Query: {query5}\n")
result5 = qa_chain_extended(query5)

print("="*70)
print("💡 ANSWER:")
print("="*70)
print(result5["result"])

print("\n" + "="*70)
print("📚 SOURCES (showing Use Cases):")
print("="*70)
for i, doc in enumerate(result5["source_documents"]):
    print(f"\n🔹 {doc.metadata['category']}: {doc.metadata['source']}")
    print(f"   {doc.page_content}")

🔄 Adding new 'Use Cases' category documents...
✅ Extended vectorstore now contains 9 documents!

❓ Query: What are some real-world applications of RAG systems?

💡 ANSWER:
Healthcare providers and e-commerce companies use RAG systems for diagnosis support and product recommendations, respectively.

📚 SOURCES (showing Use Cases):

🔹 Use Cases: Medical AI Journal
   Healthcare providers leverage RAG systems to query medical knowledge bases for diagnosis support.

🔹 AI Research: OpenAI Blog
   RAG systems improve factual accuracy by combining retrieval and generation.

🔹 Use Cases: Industry Report
   E-commerce companies use RAG to provide accurate product recommendations based on inventory data.


### Challenge 2: Experiment with Different LLM Models

OpenRouter gives you access to many models. Let's try a different one!

In [36]:
# Try a different model from OpenRouter
llm_alternative = ChatOpenAI(
    model_name="anthropic/claude-3-haiku",  # Alternative: fast Claude model
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=api_key,
    temperature=0.3
)

qa_chain_alternative = create_qa_chain(llm_alternative, retriever_extended)

print("🔄 Testing with alternative model: anthropic/claude-3-haiku\n")

query6 = "Explain how embeddings enable semantic search in RAG systems."
result6 = qa_chain_alternative(query6)

print("="*70)
print("💡 ANSWER (from Claude Haiku):")
print("="*70)
print(result6["result"])

🔄 Testing with alternative model: anthropic/claude-3-haiku

💡 ANSWER (from Claude Haiku):
Embeddings play a crucial role in enabling semantic search within RAG (Retrieval-Augmented Generation) systems. Here's how they contribute to this process:

1. Transforming text into vector representations:
   Embeddings transform the input text, whether it's a query or a passage from a knowledge base, into dense vector representations. These vector representations capture the semantic meaning and relationships between the words, phrases, or documents.

2. Capturing semantic similarity:
   The vector representations created by embeddings preserve the semantic similarity between different textual inputs. Words or passages that are semantically related will have vector representations that are closer to each other in the vector space, while unrelated inputs will have more distant vector representations.

3. Enabling semantic search:
   In a RAG system, the query provided by the user is first transfo

### Challenge 3: Implement MMR (Maximum Marginal Relevance) Retrieval

MMR promotes diversity in retrieved results, reducing redundancy.

In [37]:
# Create retriever with MMR search type
retriever_mmr = vectorstore_extended.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance
    search_kwargs={
        "k": 3,
        "fetch_k": 10,  # Fetch 10 candidates, return 3 diverse ones
        "lambda_mult": 0.5  # Balance between relevance (1.0) and diversity (0.0)
    }
)

qa_chain_mmr = create_qa_chain(llm, retriever_mmr)

print("🎯 Testing MMR retrieval for diversity...\n")

query7 = "Tell me about AI technologies and tools."
result7 = qa_chain_mmr(query7)

print("="*70)
print("💡 ANSWER (with MMR retrieval):")
print("="*70)
print(result7["result"])

print("\n" + "="*70)
print("📚 DIVERSE SOURCES RETRIEVED:")
print("="*70)
for i, doc in enumerate(result7["source_documents"]):
    print(f"\n🔹 {doc.metadata['category']}: {doc.page_content[:80]}...")

🎯 Testing MMR retrieval for diversity...

💡 ANSWER (with MMR retrieval):
AI technologies and tools mentioned include:

* OpenRouter: a unified API for accessing multiple AI models from different providers
* RAG (Relevance Aware Generator) system: a text generation system that uses text chunking strategies
* FAISS: a library for efficient vector similarity search, particularly useful for embeddings.

📚 DIVERSE SOURCES RETRIEVED:

🔹 Platform: OpenRouter provides a unified API to access multiple AI models from different pr...

🔹 Tutorial: Text chunking strategies significantly impact RAG system performance and accurac...

🔹 Database: FAISS allows efficient vector similarity search for embeddings....


### Challenge 4: Custom Prompt Template

Create a custom prompt to guide the LLM's response style.

In [38]:
# Define custom prompt template
custom_prompt = ChatPromptTemplate.from_template("""You are a helpful AI assistant specializing in RAG systems and AI technologies.
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always cite which source(s) you used in your answer.

Context: {context}

Question: {question}

Detailed Answer:""")

# Create chain with custom prompt
def create_custom_qa_chain(llm, retriever, prompt):
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    def qa_with_sources(query):
        docs = retriever.invoke(query)
        answer = rag_chain.invoke(query)
        return {"result": answer, "source_documents": docs}
    
    return qa_with_sources

qa_chain_custom = create_custom_qa_chain(llm, retriever_extended, custom_prompt)

print("🎨 Testing with custom prompt template...\n")

query8 = "What makes FAISS suitable for RAG applications?"
result8 = qa_chain_custom(query8)

print("="*70)
print("💡 ANSWER (with custom prompt):")
print("="*70)
print(result8["result"])

🎨 Testing with custom prompt template...

💡 ANSWER (with custom prompt):
FAISS (Facebook AI Similarity Search) is a library developed by Facebook AI Research (FAIR) that allows for efficient vector similarity search for embeddings. This makes it suitable for RAG (Retrieval-Augmented Generation) applications, such as searching and analyzing case law and legal precedents, as employed by legal firms.

The efficiency of FAISS in RAG applications can be attributed to its ability to:

1. **High-performance similarity search**: FAISS uses a combination of techniques, including quantization and indexing, to enable fast and efficient similarity search of dense vectors (embeddings) (Johnson et al., 2017). This is particularly useful in RAG applications where the system needs to quickly retrieve relevant documents or case law precedents from a large corpus.
2. **Scalability**: FAISS is designed to handle large-scale datasets and can efficiently search through millions of vectors (Johnson et al., 

---

## 🧾 Summary and Deliverables

### What You've Accomplished:

✅ **Part 1:** Built the foundation with LangChain components
- Created and chunked documents
- Prepared data for vectorization

✅ **Part 2:** Implemented a complete RAG pipeline
- Generated embeddings using OpenRouter
- Stored vectors in FAISS
- Created a retrieval-augmented QA system

✅ **Part 3:** Enhanced with metadata and context
- Added provenance information to documents
- Demonstrated metadata-aware retrieval
- Improved answer attribution and transparency

✅ **Bonus Challenges:** Extended the system with
- Additional document categories
- Alternative LLM models
- MMR diversity-based retrieval
- Custom prompt templates

### Key Concepts Mastered:

1. **LangChain Architecture:** Document loaders, text splitters, embeddings, vector stores, retrievers, and chains
2. **RAG Pipeline:** Embed → Store → Retrieve → Generate workflow
3. **OpenRouter Integration:** Using multiple AI providers through a unified API
4. **Metadata Management:** Enriching context with source attribution
5. **Retrieval Strategies:** Similarity search vs. MMR for different use cases
6. **Prompt Engineering:** Customizing LLM behavior with prompt templates

### Next Steps:

- 🔹 Explore different chunking strategies (semantic, recursive)
- 🔹 Try alternative vector databases (Chroma, Pinecone, Weaviate)
- 🔹 Implement re-ranking for improved retrieval quality
- 🔹 Add conversation memory for multi-turn interactions
- 🔹 Deploy your RAG system as a web application

---

## 📸 Screenshot Checklist

Capture these for your deliverables:
1. ✅ Successful embedding generation output
2. ✅ Question with answer and retrieved context
3. ✅ Metadata display showing source attribution
4. ✅ Comparison between different retrieval strategies
5. ✅ Custom prompt template results

---

**🎉 Congratulations!** You've built a production-ready RAG system from scratch using LangChain and OpenRouter!